In [1]:
import boto3
import database as db
import json
from prompters import agent_p_prompt, agent_a_prompt, agent_u_prompt, agent_l_prompt

client = boto3.client("bedrock-runtime", region_name="us-west-2")

countries = [i[0] for i in db.get_countries()]
sectors = [i[0] for i in db.get_sectors()]


def agent_p(rawBill):
    response = client.converse(
        modelId = "openai.gpt-oss-120b-1:0",
        messages = [
            {
                "role": "user",
                "content": [{"text": agent_p_prompt}]
            },
            {
                "role": "user",
                "content": [{"text": rawBill}]
            }
        ],
        inferenceConfig={
            "temperature": 0.05,
        }
    )
    return response["output"]["message"]["content"][1]["text"]


def agent_a(agent_p_res, compInfo):
    response = client.converse(
        modelId = "openai.gpt-oss-120b-1:0",
        messages = [
            {
                "role": "user",
                "content": [{"text": agent_a_prompt}]
            },
            {
                "role": "user",
                "content": [{"text": f"""Inputs:
                Bill: {agent_p_res}

                Companies information:
                {compInfo}"""}]
            }
        ],
        inferenceConfig={
            "temperature": 0.01
        }
    )
    return response["output"]["message"]["content"][1]["text"]


def agent_u(agent_p_res, agent_a_res):
    response = client.converse(
        modelId = "openai.gpt-oss-120b-1:0",
        messages = [
            {
                "role": "user",
                "content": [{"text": agent_u_prompt}]
            },
            {
                "role": "user",
                "content": [{"text": f"""Inputs:
                Bill: {agent_p_res}

                Companies information:
                {agent_a_res}"""}]
            }
        ],
        inferenceConfig={
            "temperature": 0
        }
    )
    return response["output"]["message"]["content"][1]["text"]

def agent_l(agent_p_res, agent_a_res, agent_u_res):
    response = client.converse(
        modelId = "openai.gpt-oss-120b-1:0",
        messages = [
            {
                "role": "user",
                "content": [{"text": agent_l_prompt}]
            },
            {
                "role": "user",
                "content": [{"text": f"""Inputs:
                Bill: {agent_p_res}

                Companies information:
                {agent_a_res}
                
                Impact on suppliers/clients (may not be provided for each company):{agent_u_res}"""}]
            }
        ],
        inferenceConfig={
            "temperature": 0
        }
    )
    return response["output"]["message"]["content"][1]["text"]

In [2]:
pRes = agent_p(open('cubaEmb.txt', 'r').read())
pRes = json.loads(pRes)
pRes

{'bill_profile': {'title': 'United States-Cuba Trade Act of 2021',
  'jurisdiction': 'United States',
  'status': 'enacted',
  'dates': {'published': '2021-02-04',
   'effective': '60 days after enactment',
   'expiry_or_review': ''},
  'regulatory_theme': 'sanctions',
  'summary': 'The Act repeals the long‑standing U.S. trade embargo on Cuba, removing prohibitions on exports, imports, travel, telecommunications services, remittances and other transactions. It also repeals the Cuban Democracy Act (1992) and the LIBERTAD Act (1996), amends the Trade Sanctions Reform and Export Enhancement Act, and authorizes the President to impose new export controls or emergency powers in the future. Nondiscriminatory trade treatment is extended to Cuban products, and reporting requirements are imposed on the President.'},
 'scope': {'products_or_activities': ['goods import',
   'goods export',
   'telecommunications equipment',
   'telecommunications services',
   'travel services',
   'airline opera

In [3]:
# (ticker, description, secteur)
sectors = [sector["name"] for sector in pRes["exposure_surfaces"]["sectors"]]
zones = [zone["name"] for zone in pRes["exposure_surfaces"]["countries"]]
print(sectors)
print(zones)
info1 = db.match_secteur(sectors, zones)
info2 = db.match_zone(sectors, zones)

uses = {}
for info in info2:
    uses[info[0]] = []
for info in info2:
    uses[info[0]].append(f"""{info[1]}: {info[2]}""")

q = {}
for i in info1:
    q[i[0]] = {
        "description": i[1],
        "sector": i[2],
        "operating_countries": uses[i[0]]
    }
    
agent_a_res = agent_a(pRes, q)

print(agent_a_res)

['Technology', 'Consumer Cyclical', 'Financial Services', 'Industrials']
['United States']
NVDA:  
- 4  
- positive  
- The embargo lift allows U.S. telecom equipment sales to Cuba; NVIDIA’s data‑center GPUs could be used in Cuban telecom upgrades, but market size is modest and subject to export licensing.

AVGO:  
- 5  
- positive  
- Broadcom’s networking chips and Wi‑Fi solutions become sellable to Cuban operators, creating a new export market albeit limited by regulatory approvals.

QCOM:  
- 5  
- positive  
- Qualcomm’s modem and 5G chipsets could be supplied to Cuban mobile carriers under the new trade rules, offering modest revenue upside.

DAL:  
- 7  
- positive  
- Delta can now resume scheduled passenger flights to Cuba, opening new routes and generating additional passenger and cargo revenue.

UAL:  
- 7  
- positive  
- United Airlines gains permission to operate flights to Cuban destinations, expanding its Caribbean network and ancillary revenue.

EXPE:  
- 6  
- positiv

In [4]:
agent_u_res = agent_u(pRes, agent_a_res)

In [5]:
print(agent_u_res)

**DAL** – Delta Air Lines  
- **Impact on big clients:**  
  *Delta’s existing corporate travel customers (e.g., Fortune‑500 firms, U.S. government agencies, and large travel‑management companies such as American Express GBT) will now be able to book passenger and cargo flights to Cuba for business meetings, conferences, and official visits that were previously prohibited. The removal of the embargo directly expands the itinerary options for these clients, generating incremental ticket‑sale and ancillary‑revenue opportunities.*  
  *Rationale:* The bill lifts the U.S. travel ban to Cuba, so any pre‑existing contracts that include “any destination” or “global coverage” instantly become usable for Cuba‑bound trips, increasing the value of those contracts without requiring new client acquisition.  

- **Impact on suppliers:**  
  *Delta’s core suppliers – aircraft manufacturers (Boeing, Airbus), engine providers (GE Aviation, Rolls‑Royce), fuel distributors (e.g., ExxonMobil, Chevron), an

In [6]:
agemt_l_res = agent_l(pRes, agent_a_res, agent_u_res)

In [7]:
print(agemt_l_res)

{
    "individual": {
        "DAL": {
            "exposure": "high",
            "action": "hold",
            "reason": "Delta will gain new passenger and cargo routes to Cuba, but the risk of future policy reversal warrants a cautious stance."
        },
        "UAL": {
            "exposure": "high",
            "action": "hold",
            "reason": "United can add Cuba to its network, yet potential re‑imposition of restrictions makes a conservative position prudent."
        },
        "EXPE": {
            "exposure": "medium"
        },
        "BKNG": {
            "exposure": "medium"
        },
        "ABNB": {
            "exposure": "medium"
        },
        "RCL": {
            "exposure": "medium"
        },
        "CCL": {
            "exposure": "medium"
        },
        "NCLH": {
            "exposure": "medium"
        },
        "MAR": {
            "exposure": "medium"
        },
        "NVDA": {
            "exposure": "low"
        },
        "AVGO": {


In [8]:
json.loads(agemt_l_res)

{'individual': {'DAL': {'exposure': 'high',
   'action': 'hold',
   'reason': 'Delta will gain new passenger and cargo routes to Cuba, but the risk of future policy reversal warrants a cautious stance.'},
  'UAL': {'exposure': 'high',
   'action': 'hold',
   'reason': 'United can add Cuba to its network, yet potential re‑imposition of restrictions makes a conservative position prudent.'},
  'EXPE': {'exposure': 'medium'},
  'BKNG': {'exposure': 'medium'},
  'ABNB': {'exposure': 'medium'},
  'RCL': {'exposure': 'medium'},
  'CCL': {'exposure': 'medium'},
  'NCLH': {'exposure': 'medium'},
  'MAR': {'exposure': 'medium'},
  'NVDA': {'exposure': 'low'},
  'AVGO': {'exposure': 'low'},
  'QCOM': {'exposure': 'low'},
  'JPM': {'exposure': 'low'},
  'BAC': {'exposure': 'low'},
  'WFC': {'exposure': 'low'},
  'C': {'exposure': 'low'},
  'GS': {'exposure': 'low'},
  'MS': {'exposure': 'low'},
  'V': {'exposure': 'low'},
  'MA': {'exposure': 'low'},
  'PYPL': {'exposure': 'low'},
  'NDAQ': {'expo